# <center> ***Proyecto Integrador*** </center>
## <center> ***Avance 4*** </center>
### Profesores: Dra. Grettel Barceló, Dr. Luis Enrique Falcón.
### Tutor: Dr Carlos Alberto Villaseñor Padilla
### Alumnos:

* Aurelio Antonio Lozano Rábago      A01081266


#### <p style='text-align: right;'> 30 de Mayo del 2024 </p>

Importar bibliotecas



In [10]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet, InceptionV3, VGG16, ResNet50, DenseNet121,EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
import torch
import tensorflow as tf

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Importar MobileNet base pero descartaremos las últimas capa

In [12]:
# Lista de modelos
base_models = [
    MobileNet(weights='imagenet', include_top=False),
    InceptionV3(weights='imagenet', include_top=False),
    VGG16(weights='imagenet', include_top=False),
    ResNet50(weights='imagenet', include_top=False),
    DenseNet121(weights='imagenet', include_top=False),
    EfficientNetB0(weights='imagenet', include_top=False)
]


C:\Users\44942\AppData\Local\Temp\ipykernel_16596\3195174664.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  MobileNet(weights='imagenet', include_top=False),


Crear nueva Red Neuronal y pegar redes neuronales

In [13]:
def add_custom_layers(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    preds = Dense(13, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=preds)
    return model

Cargar datos nuevos

- Carge el archivo 'train.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

Crear generador desde carpeta

In [14]:
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

train_generator = train_datagen.flow_from_directory('DataSet/organized_data/train/',
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=50,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_generator = val_datagen.flow_from_directory('DataSet/organized_data/val/',
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=25,
                                                class_mode='categorical',
                                                shuffle=True)

Found 978 images belonging to 13 classes.
Found 324 images belonging to 13 classes.


In [16]:
print(f'Número de imágenes de entrenamiento: {train_generator.samples}')
print(f'Número de imágenes de validación: {val_generator.samples}')

Número de imágenes de entrenamiento: 978
Número de imágenes de validación: 324


Compilar Modelo y entrenar desde carpeta

In [15]:
histories = []
for i, base_model in enumerate(base_models, start=1):
    print ("Modelo: ",i)
    model = add_custom_layers(base_model)

    # Congelar capas base
    for layer in base_model.layers:
        layer.trainable = False
    for layer in model.layers[-5:]:
        layer.trainable = True

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    step_size_train = train_generator.n // train_generator.batch_size
    step_size_val = val_generator.n // val_generator.batch_size

    history = model.fit(train_generator, validation_data=val_generator,
                        steps_per_epoch=step_size_train, validation_steps=step_size_val,
                        epochs=20)
    
    histories.append(history)

Modelo:  1
Epoch 1/20


c:\Users\44942\.conda\envs\new_gpu_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 506ms/step - accuracy: 0.2712 - loss: 2.1630 - val_accuracy: 0.4700 - val_loss: 1.3721
Epoch 2/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 6s 363ms/step - accuracy: 0.2600 - loss: 1.7644

c:\Users\44942\.conda\envs\new_gpu_env\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.2600 - loss: 1.7644 - val_accuracy: 0.4167 - val_loss: 1.3766
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 492ms/step - accuracy: 0.3815 - loss: 1.5110 - val_accuracy: 0.4067 - val_loss: 1.3639
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4800 - loss: 1.3899 - val_accuracy: 0.2083 - val_loss: 1.4119
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 516ms/step - accuracy: 0.3747 - loss: 1.4741 - val_accuracy: 0.4767 - val_loss: 1.3158
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5000 - loss: 1.4920 - val_accuracy: 0.4583 - val_loss: 1.3119
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 522ms/step - accuracy: 0.4291 - loss: 1.3431 - val_accuracy: 0.4933 - val_loss: 1.2695
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4000 - loss: 1.4929 - val_accuracy: 0.5833 - val_loss: 1.1968
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 544ms/step - accuracy: 0.4909 - loss: 1.2859 - val_accuracy: 0.4600 - val_

# Conclusiones

* La evaluacion de diferentes modelos pre-entrenados tienen desempenos similares, al igual que el entregable anterior, la precision y perdida de los datos de validacion son inadecuados, lo que llega a concluir que se requiere una mejor captura de datos para lograr un mejor desempeno en el entrenamiento del modelo.